<h1> Initialisation Cell</h1>

In [133]:
import numpy as np
import pandas as pd
import re

<h1> Importing the Data</h1>

In [134]:
dataset = pd.read_csv("logbook_assignment1.csv")
dataset.info()
dataset

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174870 entries, 0 to 1174869
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   date_fueled      1125700 non-null  object
 1   date_captured    1174870 non-null  object
 2   odometer         1025716 non-null  object
 3   gallons          1100660 non-null  object
 4   cost_per_gallon  1094110 non-null  object
 5   total_spent      1100660 non-null  object
 6   mpg              1100660 non-null  object
 7   miles            146274 non-null   object
 8   user_url         1174870 non-null  object
dtypes: object(9)
memory usage: 80.7+ MB


,date_fueled,date_captured,odometer,gallons,cost_per_gallon,total_spent,mpg,miles,user_url
0,"Cooling System, Heating System, Lights, Spark ...",Apr 7 2022,"73,370",NaN,NaN,NaN,NaN,NaN,https://www.somewebsite.com/suzuki/swift/2015/...
1,Nov 7 2012,Aug 30 2016,"11,983",12.120,$5.599,$67.86,31.6,NaN,https://www.somewebsite.com/bmw/x3/2009/461150
2,Sep 22 2012,Sep 28 2012,"98,233",7.991,£5.45,£43.53,28.5,NaN,https://www.somewebsite.com/mercedes-benz/e300...
3,May 4 2019,May 4 2019,"163,802",10.575,£5.11,£54.00,46.8,NaN,https://www.somewebsite.com/bmw/320d/2010/247233
4,Feb 15 2022,Feb 15 2022,NaN,11.651,$3.029,$35.29,21.0,244.4,https://www.somewebsite.com/honda/passport/201...
...,...,...,...,...,...,...,...,...,...
1174865,Feb 19 2020,Feb 19 2020,"118,237",13.046,$1.999,$26.08,25.7,NaN,https://www.somewebsite.com/volkswagen/beetle/...
1174866,Dec 6 2019,Dec 6 2019,"141,761",12.637,$2.449,$30.95,20,NaN,https://www.somewebsite.com/toyota/avalon/2000...
1174867,Oct 4 2021,Oct 4 2021,"65,029",10.002,$3.449,$34.50,28.5,NaN,https://www.somewebsite.com/fiat/500/2013/909694
1174868,Jul 26 2021,Jul 26 2021,"15,255",10.493,$3.539,$37.13,36.5,NaN,https://www.somewebsite.com/mini/cooper/2019/9...


<h1>Helper Functions</h1>

In [135]:
def formated(data):
    months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'5','Jun':'6','Jul':'07',
          'Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12','NaT':'NaT'}
    print("String has ",len(data)," chars")
    date = data.split()
    month = date[0].strip()
    day = date[1].strip()
    year = date[2].strip()
    
    date = day+"-"+months[month]+"-"+year
    return date

<h1>Data Cleaning</h1>

<h2>1.date_fueled</h2>

In [136]:
count = {}
months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'5','Jun':'6','Jul':'07',
          'Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12','NaT':'NaT'}

keys = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','NaT']
index = []

length = dataset["date_fueled"].shape[0]
for index in range(length):
    entry = dataset["date_fueled"].iloc[index]
    
    if isinstance(entry,str) == True:
        month = entry[:3]
        
        if month in count:
            count[month] += 1
            dataset["date_fueled"].iloc[index] = formated(dataset["date_fueled"].iloc[index])
            print("After = ",dataset["date_fueled"].iloc[index])
            continue
        elif month in keys:
            count[month] = 1
            dataset["date_fueled"].iloc[index] = formated(dataset["date_fueled"].iloc[index])
            print("After = ",dataset["date_fueled"].iloc[index])
            
            continue
            
        else:
            dataset["date_fueled"].iloc[index] = "NaT"
            print("After = ",dataset["date_fueled"].iloc[index])

            continue
    else:
        dataset["date_fueled"].iloc[index] = "NaT"
        print("After = ",dataset["date_fueled"].iloc[index])
        continue
        
    print("After = ",dataset["date_fueled"].iloc[index])
        
display(dataset)


After =  NaT
String has  10  chars
After =  7-11-2012
String has  11  chars
After =  22-09-2012
String has  10  chars
After =  4-5-2019
String has  11  chars
After =  15-02-2022
String has  11  chars
After =  23-10-2021
String has  11  chars
After =  20-11-2021
String has  11  chars
After =  20-01-2022
After =  NaT
After =  NaT
String has  11  chars
After =  15-11-2020
String has  11  chars
After =  14-01-2022
String has  11  chars
After =  22-11-2021
String has  11  chars
After =  28-03-2021
String has  11  chars
After =  17-11-2021
String has  10  chars
After =  1-10-2020
String has  11  chars
After =  11-5-2021
String has  11  chars
After =  30-04-2020
String has  10  chars
After =  5-07-2016
String has  11  chars
After =  11-09-2014
String has  11  chars
After =  15-5-2020
String has  10  chars
After =  3-01-2018
String has  11  chars
After =  18-6-2019
String has  10  chars
After =  1-6-2016
String has  10  chars
After =  7-11-2019
After =  NaT
After =  NaT
String has  11  chars
A

After =  4-03-2022
String has  11  chars
After =  21-09-2020
String has  10  chars
After =  2-10-2014
String has  11  chars
After =  10-03-2021
After =  NaT
String has  11  chars
After =  30-03-2018
After =  NaT
String has  10  chars
After =  5-02-2022
String has  11  chars
After =  28-09-2020
String has  10  chars
After =  7-04-2018
String has  11  chars
After =  28-09-2021
String has  10  chars
After =  2-04-2022
After =  NaT
String has  11  chars
After =  21-11-2016
String has  10  chars
After =  4-02-2021
String has  10  chars
After =  8-01-2018
After =  NaT
String has  10  chars
After =  5-03-2022
String has  11  chars
After =  12-10-2019
String has  10  chars
After =  8-09-2019
String has  11  chars
After =  25-6-2014
After =  NaT
After =  NaT
String has  11  chars
After =  17-12-2020
String has  11  chars
After =  26-04-2021
String has  11  chars
After =  15-5-2018
String has  10  chars
After =  8-12-2021
String has  10  chars
After =  9-07-2021
After =  NaT
String has  11  char

After =  NaT
String has  11  chars
After =  25-11-2019
String has  11  chars
After =  31-10-2018
String has  10  chars
After =  1-04-2022
String has  10  chars
After =  3-07-2015
String has  11  chars
After =  10-11-2020
After =  NaT
String has  11  chars
After =  27-5-2017
String has  10  chars
After =  6-08-2021
String has  11  chars
After =  26-10-2021
String has  11  chars
After =  28-08-2021
String has  11  chars
After =  18-12-2010
String has  11  chars
After =  29-5-2021
String has  11  chars
After =  21-07-2021
String has  11  chars
After =  20-08-2018
String has  11  chars
After =  17-6-2018
String has  11  chars
After =  23-01-2016
String has  11  chars
After =  29-11-2017
String has  11  chars
After =  30-12-2021
String has  11  chars
After =  18-03-2022
String has  10  chars
After =  1-12-2020
After =  NaT
String has  11  chars
After =  20-02-2019
String has  11  chars
After =  14-11-2017
String has  10  chars
After =  8-07-2011
String has  11  chars
After =  10-5-2021
Stri

KeyError: 'Marysville,'

In [ ]:
Sum = 0
for i in count.keys():
    Sum = Sum + int(count[i])

if Sum == dataset.shape[0]:
    print("All have been captured")

In [ ]:
for i in range(dataset["date_fueled"].shape[0]):
    if type(dataset["date_fueled"].iloc[i]) == type(float("nan")):
        dataset["date_fueled"].iloc[i] = "NaT"
        missing_values += 1

In [ ]:
missing_percentage = (missing_values/dataset["date_fueled"].shape[0])*100

print("The percentage of missing date fueled values = ",missing_percentage,"%")

<h2>2.Date_captured</h2>

<h2>3.Odometer</h2>

In [ ]:

length = dataset["odometer"].shape[0]
for index in range(length):
    entry = dataset["odometer"].iloc[index]
    if isinstance(entry,str):
        dataset["odometer"].iloc[index] = entry.replace(",","")
        print(dataset["odometer"].iloc[index])
        


73370 
11983 
98233 
163802 
147567 
105929 
59475 
137562 
99296 
102001 
22823 
39423 
107189 
207086 
12760 
7811 
95054 
62690 
39215 
92439 
219920 
8269 
120768 
95115 
250101 
35616 
124703 
90743 
232795 
106426 
98420 
13239 
70038 
52547 
34147 
16435 
52433 
30032 
175798 
159169 
90490 
23326 
86317 
210245 
124699 
113683 
115940 
24461 
114413 
80632 
72964 
115110 
107947 
193277 
99197 
98726 
121454 
90677 
242224 
91662 
24489 
48502 
156554 
94025 
104633 
28573 
116553 
48450 
128739 
246046 
261577 
119331 
22694 
63490 
41887 
8851 
129757 
101088 
34820 
101905 
160565 
46607 
123324 
194860 
70890 
154857 
26753 
78429 
140312 
176412 
168100 
101878 
187927 
109599 
126100 
57077 
147288 
69694 
44014 
155000 
11004 
101287 
47990 
583 
157404 
136789 
214240 
34566 
52873 
18357 
136773 
296977 
181293 
143062 
137009 
139000 
145296 
76071 
195738 
49368 
27423 
122162 
126000 
104892 
174921 
121426 
94243 
175753 
133068 
191598 
80022 
145639 
63622 
22356

108824 
122150 
238262 
1 
44746 
83595 
55863 
40331 
119546 
100065 
55531 
25728 
76213 
76414 
175749 
203308 
153596 
29925 
1440 
58694 
73322 
96338 
155900 
97985 
115629 
22612 
173981 
32862 
86171 
51255 
100133 
78527 
39211 
85899 
36602 
180362 
96519 
105346 
11193 
108614 
41589 
24306 
112995 
9589 
28368 
179836 
198741 
136176 
133405 
355957 
210252 
154116 
71632 
46095 
6567 
72380 
42806 
109838 
176049 
23635 
45010 
184302 
85501 
20223 
28724 
63361 
45486 
63488 
178544 
468681 
264838 
168402 
40409 
71243 
60877 
199776 
6367 
135181 
170906 
96470 
63433 
108446 
90446 
154008 
150976 
71351 
211223 
141861 
103731 
162855 
97859 
345 
143700 
82378 
16553 
18486 
18549 
71994 
61370 
209337 
3474 
21660 
248839 
176424 
76129 
22043 
153224 
190241 
141488 
104569 
35084 
12699 
156353 
245692 
63198 
143177 
257996 
32922 
125696 
32528 
26642 
43043 
4468 
229459 
54212 
72998 
191061 
97954 
41841 
1092 
42565 
52465 
4142 
310606 
173084 
153685 
1206

48653 
95464 
86255 
266929 
74746 
121039 
76281 
21290 
76756 
83146 
7115 
39542 
211208 
4019 
45119 
14231 
63645 
211251 
166394 
408976 
237131 
147795 
215511 
27246 
76185 
241792 
3246 
26865 
165695 
81489 
71078 
77738 
119739 
62227 
86424 
12184 
4096 
159399 
64396 
4487 
214833 
133212 
35310 
1330 
70241 
24047 
148196 
78278 
137516 
116641 
115072 
65247 
31288 
127532 
16441 
287925 
180943 
28282 
154761 
180410 
185883 
220441 
5600 
234913 
82095 
51037 
132626 
118599 
83387 
49195 
27869 
89380 
218095 
134368 
64948 
70834 
134112 
54412 
47523 
36573 
56136 
137331 
83363 
184340 
27928 
182119 
82319 
7575 
158431 
177459 
231047 
72792 
24297 
169989 
78163 
31079 
96995 
81959 
23732 
44413 
154840 
30281 
142663 
30537 
142945 
41767 
48701 
75581 
61082 
159509 
49492 
75349 
123557 
350143 
24846 
107264 
166402 
31591 
135635 
152974 
158994 
124250 
29005 
113850 
142123 
32878 
17231 
73805 
87850 
126755 
2209 
2109 
409271 
58279 
215116 
211580 
8

194130 
160173 
104688 
117057 
32866 
139982 
127177 
104034 
117344 
20618 
159430 
34797 
79510 
229249 
31291 
24703 
183257 
218150 
1338 
33049 
46902 
30251 
92290 
277372 
41119 
118002 
206259 
42253 
41438 
133095 
160581 
101160 
247611 
9229 
62758 
155372 
106128 
115241 
29551 
20828 
137087 
114935 
124120 
134479 
115899 
203287 
171164 
158036 
107197 
129397 
5077 
148613 
2788 
134440 
118705 
4837 
118403 
158455 
38762 
119316 
51023 
19411 
156377 
203052 
77962 
187897 
226000 
16058 
26301 
60088 
134666 
1344 
25984 
62728 
170725 
120492 
86509 
73179 
2929 
248987 
5666 
91389 
113911 
215481 
80075 
87572 
85574 
154553 
70153 
375450 
43244 
168181 
130694 
195100 
81748 
139716 
113490 
138551 
54139 
123696 
77142 
156011 
64370 
157214 
20817 
211363 
119454 
118865 
149322 
189446 
149871 
108747 
6651 
13505 
41771 
47232 
76420 
180668 
26045 
103839 
84285 
137674 
77709 
139833 
1275 
86826 
33538 
242077 
138373 
17672 
91708 
4144 
42465 
148050 


33458 
26630 
127053 
97585 
46240 
211149 
11689 
93031 
31159 
46881 
260691 
35001 
98427 
167022 
297826 
168641 
29414 
141453 
226841 
22522 
201632 
94383 
47896 
105362 
158836 
106491 
9600 
190206 
147032 
132939 
270049 
133989 
97821 
222469 
85819 
51810 
119126 
7220 
90067 
39948 
65815 
90326 
99187 
60120 
253227 
157995 
129375 
105076 
98317 
104617 
51220 
83556 
77484 
30934 
127495 
99491 
188490 
222489 
88718 
39506 
154230 
41804 
205714 
72568 
76375 
121215 
164607 
163532 
143431 
319429 
155366 
22135 
124579 
45604 
209360 
151180 
106283 
56337 
162691 
155756 
45590 
34501 
112408 
103834 
81434 
674557 
3255 
107442 
99736 
185762 
105120 
215704 
135192 
162610 
125749 
79169 
19013 
76596 
2912 
100098 
79660 
99531 
160982 
152454 
58154 
133652 
73794 
38726 
121750 
140716 
10583 
173697 
109893 
41895 
57601 
43744 
310685 
17240 
5966 
15377 
145112 
185753 
72863 
27297 
235250 
141423 
177832 
177735 
69760 
81257 
278663 
115275 
15933 
50458 

...

110191 
53528 
122717 
1596 
180750 
108185 
123319 
28687 
50390 
14290 
2748 
52214 
113792 
79043 
128181 
187783 
14463 
243969 
73771 
62023 
87924 
276353 
203399 
141274 
72641 
54050 
129129 
69338 
171228 
251941 
112141 
73642 
1865 
18719 
302117 
87177 
75965 
33703 
119350 
110543 
50932 
99245 
167871 
38137 
64945 
143862 
117499 
96587 
30355 
150185 
173220 
110815 
263601 
13450 
100579 
111322 
4788 
128177 
81163 
32062 
109641 
53650 
29060 
228950 
117542 
1700 
75079 
16873 
182716 
197845 
127254 
43534 
158130 
30087 
113566 
26650 
171187 
272389 
20189 
32451 
49119 
744 
120159 
49493 
44420 
115517 
72308 
161435 
19621 
161493 
229267 
152896 
16330 
49715 
234520 
69241 
63547 
156582 
146546 
311715 
127286 
142935 
199501 
75493 
32292 
76548 
3018 
71464 
131020 
31313 
130203 
190479 
150168 
127268 
39600 
230554 
63098 
185685 
146652 
47468 
31571 
124673 
147688 
142615 
61897 
182101 
1507 
57444 
209088 
198107 
220951 
23663 
191979 
41816 
858

54745 
56323 
211687 
278508 
220006 
17151 
21774 
114408 
61551 
16262 
61295 
45247 
119448 
229855 
74722 
75357 
5431 
81600 
165431 
107324 
34655 
101590 
68746 
52669 
110185 
61129 
52773 
43503 
146708 
188888 
145360 
205029 
185163 
86186 
244529 
96106 
185191 
22885 
80626 
245888 
80008 
114777 
50331 
52577 
51617 
58753 
11708 
99986 
92443 
121646 
168956 
7792 
137123 
179601 
67329 
93244 
112061 
159312 
182884 
37837 
166889 
99572 
54898 
10258 
66375 
236 
105604 
120756 
145964 
135200 
229106 
85270 
145374 
51609 
174599 
15486 
271590 
123380 
89975 
90494 
185535 
94148 
924 
72936 
102763 
118677 
140148 
147388 
58360 
73490 
134181 
22789 
222737 
73150 
65968 
97681 
199128 
149655 
160114 
260406 
19682 
39430 
90969 
70058 
10219 
16140 
2709 
59365 
63010 
74849 
280055 
44609 
10115 
161526 
53361 
130279 
11642 
19689 
371797 
113118 
46100 
221007 
66733 
150745 
167100 
146061 
70514 
149168 
176039 
116939 
99027 
33701 
121517 
102468 
183643 


140025 
132696 
108113 
76404 
1645 
109931 
80455 
271375 
17710 
265312 
133710 
51715 
43224 
17378 
23147 
110521 
48668 
36052 
30329 
81126 
61641 
104804 
270217 
160207 
77457 
102899 
164147 
118471 
53857 
42884 
124623 
164564 
166639 
71919 
9971 
34636 
34705 
222691 
111519 
183777 
98749 
49625 
27433 
124172 
169839 
139825 
77761 
82802 
116353 
123749 
7532 
104331 
60235 
231832 
156964 
39675 
34381 
168074 
89972 
72989 
94454 
75536 
100930 
68292 
72990 
116224 
49279 
100315 
58943 
7257 
35244 
43658 
126156 
46260 
20776 
53180 
41259 
178214 
17780 
102999 
150511 
114366 
153740 
126045 
95444 
64273 
164405 
106681 
158065 
1399 
72559 
260512 
245918 
9601 
254517 
124208 
2600 
181467 
107759 
220500 
87053 
20237 
165641 
69992 
132763 
80704 
40809 
62939 
207611 
73680 
93582 
168587 
139367 
154704 
85058 
122763 
45691 
56611 
117001 
22210 
204000 
141061 
277791 
90073 
16165 
210180 
3371 
160241 
57777 
68243 
90618 
56659 
69934 
34263 
95812 
1

158781 
16023 
145325 
140101 
356568 
268056 
238579 
77509 
154631 
923 
88278 
75318 
15838 
67663 
24988 
57044 
213264 
117556 
414674 
120012 
90162 
249118 
55393 
39046 
149621 
14025 
101914 
120967 
51499 
181473 
76958 
124796 
274948 
75066 
262157 
95211 
28260 
111994 
12885 
138225 
67761 
142417 
133178 
129811 
333794 
61492 
77170 
92580 
9974 
112162 
54042 
392640 
204926 
14817 
129460 
9380 
48202 
63639 
154155 
62666 
152966 
207089 
67821 
98595 
67947 
54213 
109794 
64771 
63648 
125802 
80022 
87858 
272647 
157672 
179369 
47048 
137217 
37845 
15509 
35736 
159891 
37707 
121618 
261937 
62659 
194320 
23817 
130405 
297191 
161126 
147405 
34350 
159851 
110196 
67000 
124367 
108354 
97698 
121445 
119162 
93219 
67873 
95104 
132929 
36984 
4298 
55084 
119133 
244216 
6122 
1832 
103805 
19240 
2928 
34081 
47932 
107520 
107885 
56173 
309279 
46066 
8282 
136921 
22762 
138242 
56232 
200812 
33744 
147573 
188584 
120509 
15521 
4604 
215451 
247785

159552 
109503 
192643 
117865 
106333 
64964 
146362 
81663 
26236 
37041 
171421 
156729 
62722 
164514 
115386 
56112 
27185 
106525 
58903 
193960 
59729 
202393 
184050 
37286 
79598 
87185 
7840 
147825 
69347 
37009 
188862 
36445 
26990 
204667 
144642 
98178 
122579 
112338 
65482 
199584 
355654 
67285 
28167 
127803 
138577 
4075 
206305 
136910 
87096 
174257 
140163 
209759 
29707 
74178 
273014 
125060 
35948 
57699 
11570 
106939 
202817 
152271 
25166 
20502 
81277 
173795 
235517 
115136 
90295 
82420 
207939 
109722 
34881 
188313 
141625 
198458 
121191 
181742 
54846 
114008 
95280 
12622 
104682 
107954 
42382 
13078 
112934 
172169 
1445 
64427 
121869 
104269 
194667 
138085 
44640 
134236 
104638 
60161 
40940 
80965 
184929 
154088 
211290 
21037 
133642 
82247 
153187 
213736 
8121 
195776 
169077 
100320 
12576 
125384 
20987 
31098 
133648 
160325 
65746 
78548 
86983 
182504 
33289 
73254 
51608 
82995 
36638 
12039 
110501 
291559 
156793 
9140 
76338 
208

<h2>4.Gallons</h2>

<h2>5.Cost per gallon</h2>

<h2>6.Total spent</h2>

<h2>7.Mile per gallon</h2>

<h2>8.Miles</h2>

<h1>Feature Engineering</h1>

<h1>Feature Selection</h1>